In [7]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os


batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

print(x_train.shape[1:])
nb_train_samples = int(x_train.shape[0] / 32) + 1

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch = nb_train_samples,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
(32, 32, 3)
Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 15s 10ms/step - loss: 1.8653 - acc: 0.3135 - val_loss: 1.6499 - val_acc: 0.4130
Epoch 2/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.5805 - acc: 0.4242 - val_loss: 1.3623 - val_acc: 0.5116
Epoch 3/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4610 - acc: 0.4681 - val_loss: 1.4217 - val_acc: 0.5037
Epoch 4/100
1563/1563 [==============================] - 12s 7ms/step - loss: 1.3802 - acc: 0.5029 - val_loss: 1.2115 - val_acc: 0.5698
Epoch 5/100
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3094 - acc: 0.5325 - val_loss: 1.1215 - val_acc: 0.6040
Epoch 6/100
1563/1563 [==============================] - 12s 8ms/step - loss: 1.2534 - acc: 0.5542 - val_loss: 1.1554 - val_acc: 0.5902
Epoch 7/100
1563/1563 [==============================] - 11s 7m

Epoch 60/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7930 - acc: 0.7342 - val_loss: 0.6837 - val_acc: 0.7769
Epoch 61/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7910 - acc: 0.7320 - val_loss: 0.7123 - val_acc: 0.7626
Epoch 62/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7888 - acc: 0.7314 - val_loss: 0.7214 - val_acc: 0.7608
Epoch 63/100
1563/1563 [==============================] - 12s 7ms/step - loss: 0.7930 - acc: 0.7319 - val_loss: 0.7258 - val_acc: 0.7551
Epoch 64/100
1563/1563 [==============================] - 12s 8ms/step - loss: 0.7850 - acc: 0.7368 - val_loss: 0.7879 - val_acc: 0.7435
Epoch 65/100
1563/1563 [==============================] - 12s 7ms/step - loss: 0.7905 - acc: 0.7340 - val_loss: 0.6721 - val_acc: 0.7718
Epoch 66/100
1563/1563 [==============================] - 11s 7ms/step - loss: 0.7903 - acc: 0.7359 - val_loss: 0.7428 - val_acc: 0.7584
Epoch 67/100
1563/1563 [=================